## LDA 속성분석

참고한 사이트: https://soyoung-new-challenge.tistory.com/46

In [7]:
import re
import os
import sys
import json

from konlpy.tag import Kkma
from konlpy.tag import Okt
import soynlp

import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import *
from konlpy.utils import pprint
import numpy as np
import lda
from sklearn.pipeline import Pipeline
import pandas as pd
import statsmodels.api as sm
from collections import Counter

In [8]:
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git

  Cloning https://github.com/haven-jeon/PyKoSpacing.git to /tmp/pip-req-build-ssvcr0xf
  Running command git clone -q https://github.com/haven-jeon/PyKoSpacing.git /tmp/pip-req-build-ssvcr0xf
  Using cached argparse-1.4.0-py2.py3-none-any.whl (23 kB)


In [9]:
from pykospacing import Spacing      

## 리뷰 데이터(우리가 작업한 데이터 가져오기)

In [10]:
kkma = Kkma()
okt = Okt()

In [11]:
import os
from os.path import join
import pandas as pd

data_dir = os.getenv('HOME')+'/Desktop/project/data'

df = join(data_dir, 'google_reviews_nolabel.csv')
df = pd.read_csv(df)
df.head()

,Unnamed: 0,name,ratings,date,comment,search,keyword
0,0,박성균,3,1년 전,맥주축제,세계엑스포기념품박물관,expo_science_park
1,1,Алексе́й,5,2년 전,역대 엑스포 개최 도시의 기념품과 희귀한 아이템을 한곳에서 볼 수 있습니다,세계엑스포기념품박물관,expo_science_park
2,2,Robert Helvie,3,4년 전,남북한에 관한 흥미로운 것들 거기에서 많은 시간을 보내지 않아도 됩니다 옆집 과학 ...,세계엑스포기념품박물관,expo_science_park
3,3,hyung-jun kjun,1,5년 전,엑스포는 없고 너무 지루했어,세계엑스포기념품박물관,expo_science_park
4,4,Onur Ozsoy,5,1년 전,주말에는 대개 혼잡합니다 야외 콘서트 및 엔터테인먼트 길거리 음식 작은 기념품은 지...,엑스포음악분수,expo_science_park


In [12]:
df.tail(50)

,Unnamed: 0,name,ratings,date,comment,search,keyword
8379,435,김궁금,5,3년 전,좋았네요 서울에서 가기에도 부담 없고요,계족산성,gyejok_mountain
8380,436,hyoyong lee,5,3년 전,송촌 선비마을 5단지에서 비래 사 절고개 능선 타고 계족산성 남문 산성 정상까지 왕...,계족산성,gyejok_mountain
8381,437,뭉방구아저씨,5,3년 전,간단한 운동을 하기에 좋습니다,계족산성,gyejok_mountain
8382,438,Jae eun Lee,4,3년 전,멋진 뷰,계족산성,gyejok_mountain
8383,439,장우진,4,3년 전,잘 보존된 산성,계족산성,gyejok_mountain
8384,440,김동연,4,3년 전,트레킹 코스로 정말 잘되어 있는 곳입니다 가족 모두가 트레킹 하기 좋고요 황톳길도 ...,계족산성,gyejok_mountain
8385,441,신세계,5,3년 전,대전 근거리 산행 및 트래킹 최적의 장소 다양한 코스 등,계족산성,gyejok_mountain
8386,442,김영진,5,3년 전,존멋,계족산성,gyejok_mountain
8387,443,남김덕성,4,3년 전,산책하기 좋은 장소,계족산성,gyejok_mountain
8388,444,이상묘,4,3년 전,걷기에 문경새재와 더불어 아주 좋습니다,계족산성,gyejok_mountain


In [13]:
df.isnull().sum()              # 키워드 2190개 없음

Unnamed: 0    0
name          0
ratings       0
date          0
comment       0
search        0
keyword       0
dtype: int64

In [14]:
df = df.drop(df.index[8378:8408])        # 값이 없는 행 삭제

In [15]:
df.tail()

,Unnamed: 0,name,ratings,date,comment,search,keyword
8424,480,조용래,5,4년 전,사람과 산,계족산성,gyejok_mountain
8425,481,JEFF O,4,4년 전,계족산 황톳길 한번 꼭 걸어보세요,계족산성,gyejok_mountain
8426,482,woo lee,4,4년 전,황톳길을 걸을 수 있어요,계족산성,gyejok_mountain
8427,483,JH Sung,5,5년 전,싼 자체가 흙산입니다 걷기가 아주 좋습니다 산 정상 계족산성은 복원도 잘 되어있습니다,계족산성,gyejok_mountain
8428,484,박기영,4,5년 전,대전을 볼 수 있는 좋은 장소,계족산성,gyejok_mountain


In [18]:
text = df.comment.values

In [53]:
text

array(['맥주축제', '역대 엑스포 개최 도시의 기념품과 희귀한 아이템을 한곳에서 볼 수 있습니다',
       '남북한에 관한 흥미로운 것들 거기에서 많은 시간을 보내지 않아도 됩니다 옆집 과학 공원은 한동안 수리를 위해 폐쇄되었습니다',
       ..., '황톳길을 걸을 수 있어요',
       '싼 자체가 흙산입니다 걷기가 아주 좋습니다 산 정상 계족산성은 복원도 잘 되어있습니다',
       '대전을 볼 수 있는 좋은 장소'], dtype=object)

In [21]:
df = df.drop(['name','ratings', 'date', 'keyword'], axis=1)      # 불필요한 열 제거

In [46]:
d1 = df[['comment','search']]

In [47]:
d1

,comment,search
0,맥주축제,세계엑스포기념품박물관
1,역대 엑스포 개최 도시의 기념품과 희귀한 아이템을 한곳에서 볼 수 있습니다,세계엑스포기념품박물관
2,남북한에 관한 흥미로운 것들 거기에서 많은 시간을 보내지 않아도 됩니다 옆집 과학 ...,세계엑스포기념품박물관
3,엑스포는 없고 너무 지루했어,세계엑스포기념품박물관
4,주말에는 대개 혼잡합니다 야외 콘서트 및 엔터테인먼트 길거리 음식 작은 기념품은 지...,엑스포음악분수
...,...,...
8424,사람과 산,계족산성
8425,계족산 황톳길 한번 꼭 걸어보세요,계족산성
8426,황톳길을 걸을 수 있어요,계족산성
8427,싼 자체가 흙산입니다 걷기가 아주 좋습니다 산 정상 계족산성은 복원도 잘 되어있습니다,계족산성


In [48]:
d1 = d1.set_index("search")        # 인덱스 변경

In [49]:
d1

,comment
search,
세계엑스포기념품박물관,맥주축제
세계엑스포기념품박물관,역대 엑스포 개최 도시의 기념품과 희귀한 아이템을 한곳에서 볼 수 있습니다
세계엑스포기념품박물관,남북한에 관한 흥미로운 것들 거기에서 많은 시간을 보내지 않아도 됩니다 옆집 과학 ...
세계엑스포기념품박물관,엑스포는 없고 너무 지루했어
엑스포음악분수,주말에는 대개 혼잡합니다 야외 콘서트 및 엔터테인먼트 길거리 음식 작은 기념품은 지...
...,...
계족산성,사람과 산
계족산성,계족산 황톳길 한번 꼭 걸어보세요
계족산성,황톳길을 걸을 수 있어요


In [50]:
d1 = d1.to_dict()                # 딕셔너리 변경(딕셔너리의 값이 리스트형으로 반환된다)              

In [54]:
d1

{'comment': {'세계엑스포기념품박물관': '엑스포는 없고 너무 지루했어',
  '엑스포음악분수': '야간 분수쇼가 인기지만 주변에 즐길 거리가 부족하다',
  '엑스포과학공원': '주차장 공간은 엄청 좋으나 볼 것이 없음',
  '엑스포다리': '좋은 장소는 친구 또는 가족과 함께 휴식을 취하십시오',
  '한빛탑': '대전의 상징',
  '오월드': '겨울에도 놀기 좋아요 ㅎㅎ',
  '대청호': '경치가 좋아요',
  '대청댐물문화관': '자전거 타고 운동하러 가는 곳 경치 좋은 것은 두말할 필요 없죠 다만 시끄럽게 음악 틀어놓고 영업하는 음식점은 좀 어떻게 해 줬으면 하는 바람이 몇 년이 지나도 개선이 안되네요',
  '대청호반자연생태공원': '아이들과 놀기 좋아요',
  '으능정이거리': '대전 가면 저녁 무렵 으능정이와 중앙시장 돌아보고 성심당에 들렀다가 대흥동 가서 저녁 먹고 귀가 대전 관광 코스',
  '대전스카이로드': '함 바바 좋아',
  '으능정이문화거리': '약 100 길이의 스크린이 있는 대형 몰 100',
  '성심당DCC점': '튀소 튀고 맛있다',
  '성심당대전역점': '튀긴 거라 아메리카노랑 같이 먹으면 맛납니다 느끼한 편이라 한두 개 먹으면 배불러요',
  '성심당본점': '사람이 너무 많음 ㅠ 빵은 맛은 있음',
  '성심당케잌부띠끄본점': '일단 사람이 많습니다 사람이 많다는 것은 그곳이 유명하다는 것이겠지요 롤케이크를 좋아해서 간혹 구입해서 먹는데 맛있습니다 예쁜 조각 케이크들도 많아 다양한 입맛을 충족시킬 수 있을 것입니다',
  '성심당롯데백화점대전점': '집 가까이에 성심당이 있다는 것은 좋은 일이죠',
  '계룡산국립공원수통골지구': '사계절 내내 시냇가를 보며 가볍게 걸을 수 있는 코스가 있어 너무 좋습니다',
  '수통골유원지': '좋은 산책길 간단한 등산도 가능',
  '대전예술의전당': '좋아요',
  '이응노미술관': '가족이 가서 관람하러 갔다가 입장료가 너무 비싸서',
  '대전시립미술관': 

In [55]:
comment_data = d1    # 그대로 함수를 사용하기 위해서 명 변경

## 파일을 딕셔너리 형태로 변환

In [56]:
# 파일을 사용하기 좋게 딕셔너리 형태로 변환(가져와서 쓰는 함수 그대로 쓰기 위해 일단 함수명 그대로 사용)

restaurant_review = []      

for key in comment_data.keys():          
    temp_dict={}          # 레스토랑 이름(search)을  id에 담고, 그에 해당하는 블로그 리뷰(comment)는 review에 담는다

    temp_dict['_id'] = key.split(' ')[0]   #공백으로 키를 나누고 첫번째 것을 _id에 저장 
    temp_dict['review'] = comment_data[key]   #코멘트 데이터의 키를 리뷰로 저장
    if temp_dict['review'] == []:
        continue
    restaurant_review.append(temp_dict)

In [52]:
restaurant_review

[{'_id': 'comment',
  'review': ['맥주축제',
   '역대 엑스포 개최 도시의 기념품과 희귀한 아이템을 한곳에서 볼 수 있습니다',
   '남북한에 관한 흥미로운 것들 거기에서 많은 시간을 보내지 않아도 됩니다 옆집 과학 공원은 한동안 수리를 위해 폐쇄되었습니다',
   '엑스포는 없고 너무 지루했어',
   '주말에는 대개 혼잡합니다 야외 콘서트 및 엔터테인먼트 길거리 음식 작은 기념품은 지역에서 판매됩니다 조금 더 앞에는 막대가 많은 막대의 거리가 있습니다 이곳은 특히 오후 19시 이후 매우 활기찬 곳입니다',
   '밤에 시간 잘 맞춰가면 낭만적이고 좋음',
   '19년 5월 26일을 마지막으로 종료되었습니다',
   '이젠 역사 속으로 사라질 분수 아쉽네요',
   '19년 5월 26일을 마지막으로 공사한대요 11월쯤 재오픈한다네요',
   '여름 분수 한 번쯤 가볼만 푸드트럭 다수',
   '오랜만에 봤음',
   '대전광역시',
   '이쪽은 축제가 없으면 볼거리가 없습니다',
   '한 해도 거르지 않고 찾아오는 곳 이번엔 음악분수쇼 전에 달달한 사연도 읽어주시고 먹거리 잔뜩 있어 배도 부르고 좋은 물건 싸게 살 수 있는 찬스도 아이들에겐 더 없는 천국같읏곳 어른인 나도 이 시간들이 즐겁고 행복 가득',
   '가족하고 갈만하네요',
   '음악분수와 푸드트럭 여러 가지 마켓 좋아요',
   '많은 푸드트럭과 버스킹 분수쇼를 무료로 볼 수 있는 곳',
   '음악분수의 낭만과 먹거리',
   '분수 아이가 좋아해요 먹을거리도 많고 근데 너무 더워요 ㅠㅠ열대야 어쩔 거야',
   '밤에 가면 즐거움',
   '예뻐요',
   '그래도 가끔 분위기를 살리는 행사가 있음',
   '비 와서 다 젖었어요',
   '대전 엑스포 야간 불꽃 분수쇼',
   '그나마 대전에서 갈만한 곳',
   '술 마시기도 좋고 연인과 데이트하기도 가족끼리 놀러 오기도 좋습니다',
   '여름 음악분수 엄청 멋져요',
   '2018 대전 수제 

## 함수

In [18]:
# 전처리 함수

def preprocessing(review):
    total_review = ''
    #인풋리뷰
    for idx in range(len(review)):
        r = review[idx]
        #하나의 리뷰에서 문장 단위로 자르기
        for sentence in kkma.sentences(r):
            sentence = re.sub('([a-zA-Z])','',sentence)
            sentence = re.sub('[ㄱ-ㅎㅏ-ㅣ]+','',sentence)
            sentence = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》]','',sentence)
            if len(sentence) == 0:
                continue
            if len(sentence) < 198:
                sentence = spacing(sentence)     # 띄어쓰기
            sentence += '. '
            total_review += sentence
    return total_review

In [19]:
# 감성사전 생성(추후에 변경)

service_good_feature = {'서비스':['좋','친절','괜찮','최고','빠르','짱','훌륭','추천','감사','구수','최상','대박',
                               '훈훈','특별','개이득','최고','만족','세련','최고','감동'],
                        '사장':['친절','스윗','센스'],
                        '알바':['친절','스윗','센스'],
                        '직원': ['친절','스윗','센스'],
                        '일을':['잘','빠르게'],
                        '일도':['잘','빠르게'],
                        '서빙':['잘','빠르게']}

service_bad_feature = {'서비스': ['아쉽','최악','나쁘','느리','빡치','비추','별로','그냥','낙제','쏘다쏘다','엉망','실망','불친절','문제','컴플레인',
                               '거지','그닥','그다지','구려','불편','엉성','헬','개판'],
                       '알바':['불친절','똑바로','재수'],
                       '사장':['불친절','똑바로','재수'],
                       '직원':['불친절','똑바로','재수'],
                      '일을':['못','느리게','답답'],
                      '일도':['못','느리게', '천천히'],
                      '서빙':['못','느리게','천천히','답답']}

taste_good_feature = {'간':['맞','적절','딱','환상','담백'],
                      '음식':['깔끔'],
                      '맛':['있','좋','나다','최고']}

taste_bad_feature = {'간':['세','쎄','강하다','별로'],
                     '음식':['별로','쏘다쏘다','최악'],
                     '맛':['별로','최악']}

taste_good_emotion = ['고소','부드럽','신선','촉촉','싱싱','정갈','최고']

taste_bad_emotion = ['싱겁','느끼다하다','짜다','느끼다','짜다','딱딱하다','차갑다']

cost_good_feature = {'가격': ['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다',
                            '무난','괜춘','최상','최상','굿','엄지','낮'],
                     '가성비':['괜찮','착하다','저렴','적당','싸다','좋다','합리적','훌륭','최고','만족','마음','든든','알맞다',
                            '무난','괜춘','최상','최상','굿','엄지'],
                     '양':['많','적당','푸짐하고','괜찮다','넉넉','충분','든든']
                     }

cost_bad_feature ={'가격': ['비싸','있다','나쁘','사악','비효율','높다','부담','아쉽','쏘다쏘다','별로','그닥','그다지','쎄','ㅎㄷㄷ','높','거품'],
                   '가성비':['별로'],
                   '양':['적다','작다','아쉽','적다','다소','별로'],
                   }

atmosphere_good_feature = {'분위기': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                   '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근'], 
                           '인테리어': ['좋','괜찮','조용','깔끔','적당','깡패','굉장','아담','완벽','아기자기','고급','최고','세련','만족','아늑','훌륭','예쁘','이쁘','짱',
                                    '심쿵','따뜻','깨끗','독특','매력','모던','취향저격','맘','마음','클래식','아름','인상적','귀엽','포근']}

atmosphere_bad_feature = {'분위기': ['나쁘다','바쁘다','어수선하다','이상하다','촌스럽다','별로','부담스럽다','시끄럽','복잡' ],
                          '인테리어':[]}

visit_good_feature = {'의사':['있다','충만','백프로','백프롭','많','만땅','마구','그득','만점','넘침'],
                      '다시':['가다','오다','방문','찾다','가보다','한번','갈다','찾아가다','가야지','갈거다','방문하다보고',
                            '생각나다','방문한다면','와보고','재방문','접하다','간다면','갈다때가','먹다고프다','방문한다임','오자고','가기로','갈다생각이다','가면'],
                      '굳이':[]}

visit_bad_feature = {'의사':['글쎄'],
                     '굳이':['다시','많이','여기까지','줄서서','찾아','시키다','가다','찾다','여기','기다리다','줄을','사먹'],
                     '다시':[]}

negative_word_emotion = ['안','않','못','없','아닌','아니']

In [20]:
# 특정 키워드 추출하는 함수

def get_feature_keywords(feature_keywords, review):
    feature_temp = []
    for keyword in feature_keywords:
        if re.findall(keyword, review):
            sub_list = ['게','고','음','며','데','만','도','면']
            
            for sub in sub_list:
                if sub+' ' in review:
                    review = re.sub(sub+' ', sub+',', review)
                
            a = re.findall(keyword +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K한 한 한글
            b = re.findall(keyword + '+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # K 한 한글 
            c = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+' + keyword +'[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+',review) # 한 K한글 예쁜 분위기가
                
            for ngram in a:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in b:
                t = ()
                feature_temp.append(t + (ngram,keyword))
            for ngram in c:
                t = ()
                feature_temp.append(t + (ngram,keyword))
                
    return feature_temp

In [21]:
# 특정 키워드 감정을 찾는 함수

def get_feature_emotions(feature_good_dict,feature_bad_dict, feature_temp):
    good_feature_emotion_list = []
    bad_feature_emotion_list = []
    
    for ngrams in feature_temp:
        keyword = ngrams[1]
        ngram = ngrams[0]
        is_bad_feature = None
        
        good_emotion_list = feature_good_dict[keyword]
        bad_emotion_list = feature_bad_dict[keyword]
        for emotion in good_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = False  
        for emotion in bad_emotion_list:
            if re.findall(emotion, ngram):
                is_bad_feature = True    
        for negative in negative_word_emotion:
            if re.findall(negative, ngram):
                if is_bad_feature == True:
                    is_bad_feature = False
                    break
                elif is_bad_feature == False:
                    is_bad_feature = True
                    break
                else:
                    is_bad_feature = True
                    break   
        if is_bad_feature:
            bad_feature_emotion_list.append(ngram)
        elif is_bad_feature == False:
            good_feature_emotion_list.append(ngram)
        else:
            pass
    return good_feature_emotion_list, bad_feature_emotion_list

In [22]:
# 맛에 대한 감정을 찾는 함수

def get_taste_emotion(taste_good_emotions, taste_bad_emotions):
    bad_taste_emotion_list = []
    good_taste_emotion_list = []
    for ngrams in taste_good_emotions:
        ngram = ngrams[0]
        is_bad_taste = False
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = True
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
    
    for ngrams in taste_bad_emotions:
        ngram = ngrams[0]
        is_bad_taste = True
        for negative in negative_word_emotion:
            if re.findall(negative,ngram):
                is_bad_taste = False
        if is_bad_taste:
            bad_taste_emotion_list.append(ngram)
        else:
            good_taste_emotion_list.append(ngram)
            
    return good_taste_emotion_list, bad_taste_emotion_list

In [23]:
# 최종 테스트 함수

check_division = lambda x, y: y if y ==0 else round((x / float(y)),2)

for i, restaurant in enumerate(restaurant_review):
    restaurant_good_service_count = 0
    restaurant_bad_service_count = 0
    restaurant_good_atmosphere_count =0
    restaurant_bad_atmosphere_count =0
    restaurant_good_cost_count =0
    restaurant_bad_cost_count =0
    restaurant_good_visit_count = 0
    restaurant_bad_visit_count = 0
    restaurant_good_taste_count = 0
    restaurant_bad_taste_count = 0
    print(i, restaurant['_id'])
    reviews_list = restaurant
    for j, review in enumerate(reviews_list):
        service_temp = get_feature_keywords(service_good_feature.keys(), review)
        good_service,bad_service = get_feature_emotions(service_good_feature, service_bad_feature, service_temp)

        atmosphere_temp = get_feature_keywords(atmosphere_good_feature.keys(), review)
        good_atmosphere,bad_atmosphere = get_feature_emotions(atmosphere_good_feature, atmosphere_bad_feature, atmosphere_temp)

        cost_temp = get_feature_keywords(cost_good_feature.keys(), review)
        good_cost,bad_cost = get_feature_emotions(cost_good_feature, cost_bad_feature, cost_temp)

        visit_temp = get_feature_keywords(visit_good_feature.keys(), review)
        good_visit,bad_visit = get_feature_emotions(visit_good_feature, visit_bad_feature, visit_temp)

        taste_temp = get_feature_keywords(taste_good_feature.keys(), review)
        good_taste,bad_taste = get_feature_emotions(taste_good_feature, taste_bad_feature, taste_temp)
        taste_good_emotion_temp = get_feature_keywords(taste_good_emotion, review)
        taste_bad_emotion_temp = get_feature_keywords(taste_bad_emotion, review)
        good_taste2, bad_taste2 = get_taste_emotion(taste_good_emotion_temp,taste_bad_emotion_temp)
        good_taste.extend(good_taste2)
        bad_taste.extend(bad_taste2)

        if len(good_service) > len(bad_service):
            restaurant_good_service_count += 1
        elif len(good_service) < len(bad_service):
            restaurant_bad_service_count += 1
        else:
            pass
        
        if len(good_atmosphere) > len(bad_atmosphere):
            restaurant_good_atmosphere_count += 1
        elif len(good_atmosphere) < len(bad_atmosphere):
            restaurant_bad_atmosphere_count += 1
        else:
            pass
        
        if len(good_cost) > len(bad_cost):
            restaurant_good_cost_count += 1
        elif len(good_cost) < len(bad_cost):
            restaurant_bad_cost_count += 1
        else:
            pass
            
        if len(good_visit) > len(bad_visit):
            restaurant_good_visit_count += 1
        elif len(good_visit) < len(bad_visit):
            restaurant_bad_visit_count += 1
        else:
            pass
        
        if len(good_taste) > len(bad_taste):
            restaurant_good_taste_count += 1
        elif len(good_taste) < len(bad_taste):
            restaurant_bad_taste_count += 1
        else:
            pass
        
    TT = restaurant_good_service_count + restaurant_bad_service_count + restaurant_good_taste_count + restaurant_bad_taste_count + restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count + restaurant_good_cost_count + restaurant_bad_cost_count
    
    if TT > 5:
        print('Total review count: {}'.format(len(reviews_list)))
        print('Good service: {}/{} = {}'.format(restaurant_good_service_count,restaurant_good_service_count + restaurant_bad_service_count,100*check_division(restaurant_good_service_count, restaurant_good_service_count + restaurant_bad_service_count)))
        print('Good atmosphere: {}/{} = {}'.format(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count,100*check_division(restaurant_good_atmosphere_count,restaurant_good_atmosphere_count + restaurant_bad_atmosphere_count))) 
        print('Good cost: {}/{} = {}'.format(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count, 100*check_division(restaurant_good_cost_count,restaurant_good_cost_count + restaurant_bad_cost_count)))
        print('Good taste: {}/{} = {}'.format(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count,100*check_division(restaurant_good_taste_count,restaurant_good_taste_count + restaurant_bad_taste_count)))
        print('')

0 comment
1 search
